<h1>Surfcast.com</h1>
<h3>A Goodfellow Analytics Creation</h3>
<h5>In partnership with Griffin Global</h5>

In [1]:
# Reset Notebook
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
# Import python packages
from bs4 import BeautifulSoup
from urllib import request
import requests
from urllib.request import urlopen
from datetime import datetime, timedelta
from pytz import timezone
import pytz
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser
from dateutil import tz
import scipy.interpolate
import os
from sqlalchemy import create_engine # database connection

# Matplotlib inline
%matplotlib inline

# Hide ipython notebook warnings
import warnings
warnings.filterwarnings('ignore')

<h1>Step 1</h1>
<h3>Get HTML Database as List of Files</h3>

In [14]:
# Set URL path to NOAA 'gridded fields' database
url = 'http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridded_fields/'

In [15]:
# Set URL path to NOAA 'gridded fields' map files
url_map = 'http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridded_fields/map_files/'

In [16]:
# File types of interest [wave, wind, surface current, surface temperature]
#extension_list = ['wav', 'wnd', 'cur', 'swt', 'ice', 'o', 'e', 's', 'm', 'h']
extension_list = ['wav', 'wnd', 'cur', 'swt', 'ice', 'o']

In [17]:
# Define NOAA database class
class NoaaDB:
    
    """
    Class: NoaaDB
        - This class converts the NCAST|FCAST FTP database list into a pandas DataFrame
        
        - The gridded fields filename format is:

          LYYYYDDDHH.N.EXT

          L    = lake letter (s=Superior, m=Michigan, h=Huron, e=Erie, o=Ontario)
          YYYY = year at start of simulation (GMT)
          DDD  = Day Of Year at start of simulation (GMT)
          HH   = hr at start of simulation (GMT)
          N    = Site Number
    """

    # Initialize object
    def __init__(self, url, extension_list):
        
        # Set object attributes
        self.url = url                            # FTP 'gridded data' database URL
        self.url_ncast = self.url  + 'NCAST/'     # FTP NCAST database URL  
        self.url_fcast = self.url  + 'FCAST/'     # FTP FCAST database URL   
        self.extension_list = extension_list      # List of file type extensions   
        self.html_ncast = {}                      # HTML from FCAST page
        self.html_fcast = {}                      # HTML from FCAST page
        self.html_obj_ncast = {}                  # FCAST HTML Object
        self.html_obj_fcast = {}                  # FCAST HTML Object
        
        # Current UTC time as GMT
        self.current_datetime_GMT = datetime.utcnow().replace(tzinfo=tz.gettz('GMT'))
        
        # Database dataframe
        self.df = pd.DataFrame(index=[], columns=['filename', 'file_extension', 
                                                  'filetype', 'lake', 'file_datetime', 
                                                  'current_datetime','forecast_type', 'file_url'])                                       
        
    # Get NCAST database 
    def get_ncast(self):
        
        # Get DataFrame row count
        df_rows = self.df.shape[0]-1
        
        # Get HTML from database page
        self.html_ncast   = requests.get(self.url_ncast)
        
        # Create BeautifulSoup object
        self.html_obj_ncast   = BeautifulSoup(self.html_ncast.content)
        
        # Set database list as dataframe 
        for link in self.html_obj_ncast.findAll('a', href=True):
            if (
                link.contents[0].split('.')[-1] in self.extension_list and 
                link.contents[0][0] in self.extension_list
               ):
                
                df_rows += 1  # row count                                                 
                
                filename = link.contents[0]                                         # file name
                file_extension = link.contents[0].split('.')[-1]                    # file extension
                
                # file datetime as GMT
                file_datetime = datetime.strptime(link.contents[0].split('.')[0][1:len(link.contents[0].split('.')[0])], "%Y%j%H")       # file datetime (GMT)
                file_datetime = file_datetime.replace(tzinfo=tz.gettz('GMT'))
                
                # Set file type
                if file_extension == 'wav':           # Wave
                    filetype = 'WAVES'
                elif file_extension == 'wnd':         # Wind
                    filetype = 'WINDS'
                elif file_extension == 'cur':         # Surface Current
                    filetype = 'SURFACE CURRENTS'
                elif file_extension == 'swt':         # Surface Temperature
                    filetype = 'SURFACE TEMPS'
                elif file_extension == 'ice':         # Ice Conditions
                    filetype = 'ICE PARAMS'
                
                # Set great lake
                if filename[0] == 'e':       # Lake Erie
                    lake = 'erie'
                elif filename[0] == 'h':     # Lake Huron
                    lake = 'huron'
                elif filename[0] == 'o':     # Lake Ontario
                    lake = 'ontario'
                elif filename[0] == 's':     # Lake Superior
                    lake = 'superior'
                elif filename[0] == 'm':     # Lake Michigan
                    lake = 'michigan'            
                
                # save to dataframe
                self.df.loc[df_rows] = [filename, file_extension, filetype, 
                                        lake, file_datetime, self.current_datetime_GMT, 
                                        'NCAST', self.url_ncast]                                   
                          
    # Get FCAST database 
    def get_fcast(self):
        
        # Get DataFrame row count
        df_rows = self.df.shape[0]-1
        
        # Get HTML from database page
        self.html_fcast   = requests.get(self.url_fcast)
        
        # Create BeautifulSoup object
        self.html_obj_fcast   = BeautifulSoup(self.html_fcast.content)
        
        # Set database list as dataframe 
        for link in self.html_obj_fcast.findAll('a', href=True):
            if (
                link.contents[0].split('.')[-1] in self.extension_list and 
                link.contents[0][0] in self.extension_list
               ):
                
                df_rows += 1  # row count                                                 
                
                filename = link.contents[0]                                         # file name
                file_extension = link.contents[0].split('.')[-1]                    # file extension
                
                # file datetime as GMT
                file_datetime = datetime.strptime(link.contents[0].split('.')[0][1:len(link.contents[0].split('.')[0])], "%Y%j%H")       # file datetime (GMT)
                file_datetime = file_datetime.replace(tzinfo=tz.gettz('GMT'))
                
                # Set file type
                if file_extension == 'wav':           # Wave
                    filetype = 'WAVES'
                elif file_extension == 'wnd':         # Wind
                    filetype = 'WINDS'
                elif file_extension == 'cur':         # Surface Current
                    filetype = 'SURFACE CURRENTS'
                elif file_extension == 'swt':         # Surface Temperature
                    filetype = 'SURFACE TEMPS'
                elif file_extension == 'ice':         # Ice Conditions
                    filetype = 'ICE PARAMS'
                
                # Set great lake
                if filename[0] == 'e':       # Lake Erie
                    lake = 'erie'
                elif filename[0] == 'h':     # Lake Huron
                    lake = 'huron'
                elif filename[0] == 'o':     # Lake Ontario
                    lake = 'ontario'
                elif filename[0] == 's':     # Lake Superior
                    lake = 'superior'
                elif filename[0] == 'm':     # Lake Michigan
                    lake = 'michigan'       
                
                # save to dataframe
                self.df.loc[df_rows] = [filename, file_extension, filetype, 
                                        lake, file_datetime, self.current_datetime_GMT, 
                                        'FCAST', self.url_fcast]

In [18]:
# Create NoaaDB object
noaa_files = NoaaDB(url, extension_list)

In [19]:
# Get NCAST files 
noaa_files.get_ncast()

In [20]:
# Get FCAST files 
noaa_files.get_fcast()

In [21]:
# Show Dataframe of NoaaDB files 
noaa_files.df.head()

,filename,file_extension,filetype,lake,file_datetime,current_datetime,forecast_type,file_url
0,o201611706.0.ice,ice,ICE PARAMS,ontario,2016-04-26 06:00:00,2016-05-28 16:33:26.739723,NCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
1,o201611712.0.ice,ice,ICE PARAMS,ontario,2016-04-26 12:00:00,2016-05-28 16:33:26.739723,NCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
2,o201611718.0.ice,ice,ICE PARAMS,ontario,2016-04-26 18:00:00,2016-05-28 16:33:26.739723,NCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
3,o201611800.0.ice,ice,ICE PARAMS,ontario,2016-04-27 00:00:00,2016-05-28 16:33:26.739723,NCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
4,o201611806.0.ice,ice,ICE PARAMS,ontario,2016-04-27 06:00:00,2016-05-28 16:33:26.739723,NCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...


In [22]:
# Show Dataframe of NoaaDB files 
noaa_files.df.tail()

,filename,file_extension,filetype,lake,file_datetime,current_datetime,forecast_type,file_url
219,o201614900.0.wnd,wnd,WINDS,ontario,2016-05-28 00:00:00,2016-05-28 16:33:26.739723,FCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
220,o201614912.0.cur,cur,SURFACE CURRENTS,ontario,2016-05-28 12:00:00,2016-05-28 16:33:26.739723,FCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
221,o201614912.0.swt,swt,SURFACE TEMPS,ontario,2016-05-28 12:00:00,2016-05-28 16:33:26.739723,FCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
222,o201614912.0.wav,wav,WAVES,ontario,2016-05-28 12:00:00,2016-05-28 16:33:26.739723,FCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...
223,o201614912.0.wnd,wnd,WINDS,ontario,2016-05-28 12:00:00,2016-05-28 16:33:26.739723,FCAST,http://www.glerl.noaa.gov/ftp/EMF/glcfs/gridde...


<h1>Step 2</h1>
<h3>Find Most Recently Uploaded Files</h3>

In [ ]:
# Define NOAA database file class
class NoaaDB_NewestFile():
    
    """
    Class: NoaaDB_NewestFile
        - This class Takes a lake and attribute as input and finds the most recently uploaded corresponding file. 
    
    Inputs:
            - lake:       Lake of interest [ontario, michigan, erie, superior, huron] 
            - attribute:  File extension [cur, swt, wav, wnd] 
    """

    # Initialize object
    def __init__(self, noaa_files, url_map):
        
        # Set object attributes
        self.noaa_files = noaa_files              # user input NoaaDB object (Pandas Dataframe of all files in Noaa DB)
        self.df_ncast = {}                        # downloaded NCAST text files as DataFrame
        self.df_fcast = {}                        # downloaded FCAST text files as DataFrame
        self.df_ncast_header = {}                 # NCAST text file headers
        self.df_fcast_header = {}                 # FCAST text file headers
        self.df = {}                              # conbined NCAST and FCAST DataFrames of most recent 120 hr forecast
        self.url_map = url_map                    # url containing map files
        
        # Set map path
        self.map_path = r'C:\Users\Sebastian\Projects\Websites\Surfcast\GetData\GridFiles'
        
        # Get Newest NCAST Files in database and save as DataFrame
        maxtime = noaa_files.df[(noaa_files.df.forecast_type == 'NCAST')]['file_datetime'].max()
        self.newest_files_ncast = noaa_files.df[(noaa_files.df.forecast_type == 'NCAST') &
                                                (noaa_files.df.file_datetime == maxtime)].reset_index(drop=True)
        
        # Get Newest FCAST Files in database and save as DataFrame
        maxtime = noaa_files.df[(noaa_files.df.forecast_type == 'FCAST')]['file_datetime'].max()
        self.newest_files_fcast = noaa_files.df[(noaa_files.df.forecast_type == 'FCAST') & 
                                                (noaa_files.df.file_datetime == maxtime)].reset_index(drop=True)

        
        
        
    # Find the number of grid points for a given file
    def grid_count(self, row):

        # Get first line in file
        first_line = urlopen(row['file_url'] + row['filename']).readline()

        # Get grid number from first line
        grid_num = first_line.split()[-1].decode('ascii')

        return grid_num
    
    
    
    
    
    
    # Download newest file and save as row delimited list
    def download_file(self, url, filename):

        """
        Function: download_file
            - This function will download from the NOAA database the text file corresponding to the filename
              and url input by the user and return a row delimited text file.
              
        Inputs:
            - filename:     Name of the file to download
            - url:          Url of the file to download 

        Outputs:
            - textfile:     The corresponding row delimited text file
        """           
        
        # Send file request to server and download
        response = request.urlopen(url + filename)

        # Parse text file by line breaks
        textfile = str(response.read().decode('utf-8')).split('\n')
        
        # return row delimited text file
        return textfile
    
    
    
    
    # Get the number of forecast hours per file
    def hour_count(self, url, filename):

        text_file = self.download_file(url, filename)

        hours = 0    # number of data rows in text file
        
        # Loop through all rows in file
        for row in text_file:
            if 'dat' in row:
                hours += 1  # hour count

        # Return number of forecast hours          
        return hours
    
    
    def get_headers(self):
        
        # Define function to extract header information
        def header_array(row):
            if 'dat' in row and len(row.split()) != 0:
                array = np.array(row.split()).astype(object)
                return array
            else: 
                return None
            
        header_array_vec = np.vectorize(header_array)  # Vecotize function
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # NCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        self.df_ncast_header = pd.DataFrame(index=[0], 
                                            columns=['year_start', 'day_start', 'hour_start', 'datetime_start', 'skip_start', 
                                                     'year_end', 'day_end', 'hour_end', 'datetime_end', 'skip_end', 
                                                     'grid_count', 'forecast_hours', 'data_rows',  
                                                     'lake', 'filetype'])

        # Write NCAST data to DataFrame
        row_count = 0
        for lake in self.newest_files_ncast.lake.unique():
            
            df_lake = self.newest_files_ncast[(self.newest_files_ncast.lake == lake)]

            for df_index in df_lake.index:
                
                # Download text file
                text_file = self.download_file(self.newest_files_ncast.file_url[df_index], 
                                               self.newest_files_ncast.filename[df_index])
                
                text_header = header_array_vec(text_file)
                text_header = text_header[text_header != np.array(None)]
                text_header = np.vstack(text_header)
                
                self.df_ncast_header.loc[row_count, ['year_start', 'day_start', 'hour_start']] = text_header[0, 0:3]
                file_datetime = datetime.strptime(self.df_ncast_header.year_start[row_count] + 
                                                  self.df_ncast_header.day_start[row_count] + 
                                                  self.df_ncast_header.hour_start[row_count], "%Y%j%H")       
                self.df_ncast_header.loc[row_count, ['datetime_start']] = file_datetime
                
                self.df_ncast_header.loc[row_count, ['year_end', 'day_end', 'hour_end']] = text_header[-1, 0:3]
                file_datetime = datetime.strptime(self.df_ncast_header.year_end[row_count] + 
                                                  self.df_ncast_header.day_end[row_count] + 
                                                  self.df_ncast_header.hour_end[row_count], "%Y%j%H")       
                self.df_ncast_header.loc[row_count, ['datetime_end']] = file_datetime
                
                self.df_ncast_header.loc[row_count, ['grid_count']] = int(text_header[0, -1])
                self.df_ncast_header.loc[row_count, ['forecast_hours']] = int(text_header.shape[0])
                self.df_ncast_header.loc[row_count, ['data_rows']] = self.df_ncast_header.grid_count[row_count] * \
                                                                     self.df_ncast_header.forecast_hours[row_count]
                self.df_ncast_header.loc[row_count, ['lake']] = self.newest_files_ncast.lake[df_index]
                self.df_ncast_header.loc[row_count, ['filetype']] = self.newest_files_ncast.filetype[df_index]
                
                row_count += 1 
                
        # -------------------------------------------------------------------------------------------------------------------- #
        # FCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        self.df_fcast_header = pd.DataFrame(index=[0], 
                                            columns=['year_start', 'day_start', 'hour_start', 'datetime_start', 'skip_start', 
                                                     'year_end', 'day_end', 'hour_end', 'datetime_end', 'skip_end', 
                                                     'grid_count', 'forecast_hours', 'data_rows',  
                                                     'lake', 'filetype'])

        # Write FCAST data to DataFrame
        row_count = 0
        for lake in self.newest_files_fcast.lake.unique():
            
            df_lake = self.newest_files_fcast[(self.newest_files_fcast.lake == lake)]

            for df_index in df_lake.index:
                
                # Download text file
                text_file = self.download_file(self.newest_files_fcast.file_url[df_index], 
                                               self.newest_files_fcast.filename[df_index])
                
                text_header = header_array_vec(text_file)
                text_header = text_header[text_header != np.array(None)]
                text_header = np.vstack(text_header)
                
                self.df_fcast_header.loc[row_count, ['year_start', 'day_start', 'hour_start']] = text_header[0, 0:3]
                file_datetime = datetime.strptime(self.df_fcast_header.year_start[row_count] + 
                                                  self.df_fcast_header.day_start[row_count] + 
                                                  self.df_fcast_header.hour_start[row_count], "%Y%j%H")       
                self.df_fcast_header.loc[row_count, ['datetime_start']] = file_datetime
                
                self.df_fcast_header.loc[row_count, ['year_end', 'day_end', 'hour_end']] = text_header[-1, 0:3]
                file_datetime = datetime.strptime(self.df_fcast_header.year_end[row_count] + 
                                                  self.df_fcast_header.day_end[row_count] + 
                                                  self.df_fcast_header.hour_end[row_count], "%Y%j%H")       
                self.df_fcast_header.loc[row_count, ['datetime_end']] = file_datetime
                
                self.df_fcast_header.loc[row_count, ['grid_count']] = int(text_header[0, -1])
                self.df_fcast_header.loc[row_count, ['forecast_hours']] = int(text_header.shape[0])
                self.df_fcast_header.loc[row_count, ['data_rows']] = self.df_fcast_header.grid_count[row_count] * \
                                                                     self.df_fcast_header.forecast_hours[row_count]
                self.df_fcast_header.loc[row_count, ['lake']] = self.newest_files_fcast.lake[df_index]
                self.df_fcast_header.loc[row_count, ['filetype']] = self.newest_files_fcast.filetype[df_index]
                
                row_count += 1 

    # Add grid information to newest_files_ncast and newest_files_fcast
    def add_grid_data(self):
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # NCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        # Get number of grid points for each file
        self.newest_files_ncast['grid_count'] = pd.Series(index=self.newest_files_ncast.index)
        self.newest_files_ncast['grid_count'] = self.newest_files_ncast.apply(self.grid_count, axis=1).astype(int)
        
        # Get hours per file
        hours_ncast = self.hour_count(self.newest_files_ncast.file_url[0], 
                                      self.newest_files_ncast.filename[0])
        
        self.newest_files_ncast['forecast_hours'] = pd.Series(index=self.newest_files_ncast.index)  
        self.newest_files_ncast['forecast_hours'] = hours_ncast                                  
        
        # Set data rows
        self.newest_files_ncast['data_rows'] = pd.Series(index=self.newest_files_ncast.index)
        self.newest_files_ncast['data_rows'] = self.newest_files_ncast['grid_count'] * \
                                               self.newest_files_ncast['forecast_hours']     
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # FCAST 
        # -------------------------------------------------------------------------------------------------------------------- #

        # Get number of grid points for each file
        self.newest_files_fcast['grid_count'] = pd.Series(index=self.newest_files_fcast.index)
        self.newest_files_fcast['grid_count'] = self.newest_files_fcast.apply(self.grid_count, axis=1).astype(int) 

        # Get hours per file
        hours_fcast = self.hour_count(self.newest_files_fcast.file_url[0], 
                                      self.newest_files_fcast.filename[0])
        
        self.newest_files_fcast['forecast_hours'] = pd.Series(index=self.newest_files_fcast.index)  
        self.newest_files_fcast['forecast_hours'] = hours_fcast                                    

        # Set data rows
        self.newest_files_fcast['data_rows'] = pd.Series(index=self.newest_files_fcast.index)
        self.newest_files_fcast['data_rows'] = self.newest_files_fcast['grid_count'] * \
                                               self.newest_files_fcast['forecast_hours']     
            
            
            
            
    # Set up empty DataFrames to hold text file data
    def df_setup(self):
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # NCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        # Get number of NCAST rows
        row_num = self.newest_files_ncast.data_rows.unique().sum()

        # NOAA attributes
        wave =        ['wave_height', 'wave_direction', 'wave_period']
        wind =        ['wind_speed', 'wind_direction']
        temperature = ['surface_temperature']
        current =     ['currect_speed', 'current_direction']
        ice =         ['ice_concentration', 'ice_thickness', 'ice_speed', 'ice_direction']
        
        # Get list of unique NCAST file types and lakes
        filetype_ncast = self.newest_files_ncast.filetype.unique()
        
        # Setup NCAST DataFrame
        self.df_ncast  = pd.DataFrame(index=range(row_num), 
                                      columns=['year', 'day', 'hour', 'datetime',                       # datetime 
                                               'grid_number', 'latitude', 'longitude', 'map', 'lake'])  # grid    
        
        if any('WAVES' in s for s in filetype_ncast):               # wave
            for col in wave:
                self.df_ncast[col] = pd.Series(index=self.df_ncast.index) 
                
        if any('WINDS' in s for s in filetype_ncast):               # wind              
            for col in wind:
                self.df_ncast[col] = pd.Series(index=self.df_ncast.index)
                
        if any('SURFACE TEMPS' in s for s in filetype_ncast):       # temperature
            for col in temperature:
                self.df_ncast[col] = pd.Series(index=self.df_ncast.index)
                
        if any('SURFACE CURRENTS' in s for s in filetype_ncast):    # current
            for col in current:
                self.df_ncast[col] = pd.Series(index=self.df_ncast.index)
                
        if any('ICE PARAMS' in s for s in filetype_ncast):          # ice
            for col in ice:
                self.df_ncast[col] = pd.Series(index=self.df_ncast.index)
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # FCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        # Get number of NCAST rows
        row_num = self.newest_files_fcast.data_rows.unique().sum()

        # NOAA attributes
        wave =        ['wave_height', 'wave_direction', 'wave_period']
        wind =        ['wind_speed', 'wind_direction']
        temperature = ['surface_temperature']
        current =     ['currect_speed', 'current_direction']
        ice =         ['ice_concentration', 'ice_thickness', 'ice_speed', 'ice_direction']
        
        # Get list of unique NCAST file types and lakes
        filetype_fcast = self.newest_files_fcast.filetype.unique()

        # Setup NCAST DataFrame
        self.df_fcast  = pd.DataFrame(index=range(row_num), 
                                      columns=['year', 'day', 'hour', 'datetime',                       # datetime 
                                               'grid_number', 'latitude', 'longitude', 'map', 'lake'])  # grid    
        
        if any('WAVES' in s for s in filetype_fcast):                         # wave
            for col in wave:
                self.df_fcast[col] = pd.Series(index=self.df_fcast.index) 
                
        if any('WINDS' in s for s in filetype_fcast):                         # wind              
            for col in wind:
                self.df_fcast[col] = pd.Series(index=self.df_fcast.index)
                
        if any('SURFACE TEMPS' in s for s in filetype_fcast):                 # temperature
            for col in temperature:
                self.df_fcast[col] = pd.Series(index=self.df_fcast.index)
                
        if any('SURFACE CURRENTS' in s for s in filetype_fcast):              # current
            for col in current:
                self.df_fcast[col] = pd.Series(index=self.df_fcast.index)
                
        if any('ICE PARAMS' in s for s in filetype_fcast):                    # ice
            for col in ice:
                self.df_fcast[col] = pd.Series(index=self.df_fcast.index)
                
                
                
                
    # Fill NCAST and FCAST DataFrames with text file data
    def df_fill(self):
        
        """
        The gridded fields data format is:

          col_1 = grid number corresponding to a grid file
        
        For Wind Speed:
          col_2 = 10m wind speed at grid center (m/s)
          col_3 = wind direction at grid center (0 = from north, 90 = from east)

        For Surface Currents:
          col_2 = surface current speed at grid center (m/s)
          col_3 = surface current direction at grid center (0 = toward north, 90 = toward east)

        For Waves:
          col_2 = significant wave height at grid center (m)
          col_3 = significant wave direction at grid center (0 = toward north, 90 = toward east)
          col_4 = significant wave period (s)

        For Surface Water Temps:
          col_2 = surface water temperature (C) at grid center

        For Ice Model results:
          col_2 = ice concentration (0-1) at grid centerwww.f
          col_3 = ice thickness (m) at grid center
          col_4 = ice speed (m/s) at grid cente
          col_5 = ice direction (m/s) at grid center
        """

        # Define function to parse text file rows and identify headers
        def data_array(row):
            if 'dat' not in row and len(row.split()) != 0:
                array = np.array(row.split()).astype(float)
                return array
            else: 
                return None
            
        data_array_vec = np.vectorize(data_array)  # Vecotize function
        
        # Define function to extract header information
        def header_array(row):
            if 'dat' in row and len(row.split()) != 0:
                array = np.array(row.split()).astype(object)
                return array
            else: 
                return None
            
        header_array_vec = np.vectorize(header_array)  # Vecotize function
        
        # NOAA attributes
        wave =        ['grid_number', 'wave_height', 'wave_direction', 'wave_period']
        wind =        ['grid_number', 'wind_speed', 'wind_direction']
        temperature = ['grid_number', 'surface_temperature']
        current =     ['grid_number', 'currect_speed', 'current_direction']
        ice =         ['grid_number', 'ice_concentration', 'ice_thickness', 'ice_speed', 'ice_direction']
        
        # -------------------------------------------------------------------------------------------------------------------- #
        # NCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        # Write NCAST data to DataFrame
        row_count = 0
        for lake in self.newest_files_ncast.lake.unique():
            
            df_lake = self.newest_files_ncast[(self.newest_files_ncast.lake == lake)]

            for df_index in df_lake.index:
                
                # Set attributes to set based on filetype
                if  self.newest_files_ncast.filetype[df_index] == 'WAVES':
                    cols = wave
                elif self.newest_files_ncast.filetype[df_index] == 'WINDS':
                    cols = wind
                elif self.newest_files_ncast.filetype[df_index] == 'SURFACE TEMPS':
                    cols = temperature
                elif self.newest_files_ncast.filetype[df_index] == 'SURFACE CURRENTS':
                    cols = current
                elif self.newest_files_ncast.filetype[df_index] == 'ICE PARAMS':
                    cols = ice   
                
                # Download text file
                text_file = self.download_file(self.newest_files_ncast.file_url[df_index], 
                                               self.newest_files_ncast.filename[df_index])
                
                # Get map from header
                map = text_file[0].split()[3].split('/')[-1]
                map = map.split('.')[0] + '.' + 'map'
                if lake == 'superior':
                    map = 'superior' + map.split('sup')[1]
                
                # Format text file data as numpy array
                text_data = data_array_vec(text_file)                      # parse rows
                text_data = text_data[text_data != np.array(None)]         # remove headers
                text_data = np.vstack(text_data)
                
                # Format text file header data as numpy array
                text_header = header_array_vec(text_file)
                text_header = text_header[text_header != np.array(None)]
                text_header = np.vstack(text_header)
                
                print('')
                print('NCAST HEADER')
                print(text_header)
                print('')

                # Create empty arrays for time (y, d, r) and datetime (GMT)
                time_count = np.zeros([self.newest_files_ncast.data_rows[df_index] ,3]).astype(int)
                date_time = np.zeros([self.newest_files_ncast.data_rows[df_index] ,1]).astype(object)
                
                # set row count
                count = 0 
                
                # Loop thought text file headers
                for header in text_header:
                    
                    # Set year, day, hour
                    time_count[count:count + self.newest_files_ncast.grid_count[df_index], 0] = header[0]
                    time_count[count:count + self.newest_files_ncast.grid_count[df_index], 1] = header[1]
                    time_count[count:count + self.newest_files_ncast.grid_count[df_index], 2] = header[2]
                    
                    # Set date time object (GMT)
                    date_time[count:count + self.newest_files_ncast.grid_count[df_index], 0] = datetime.strptime(header[0] + header[1] + header[2], "%Y%j%H") 
                    
                    # Update count
                    count = count + self.newest_files_ncast.grid_count[df_index]
          
                # Set text file data as DataFrame
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, cols] = text_data
                
                # Set header year, day, hour
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'year'] = time_count[:, 0]
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'day'] = time_count[:, 1]
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'hour'] = time_count[:, 2]
                
                # Set date time object (GMT)
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'datetime'] = date_time
                
                # Set static values
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'map'] = map    # map
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 'lake'] = lake  # lake
                
                # Get map grid data
                map_file = self.download_file(url_map, map)
                map_data = np.loadtxt(map_file)
                map_data = np.tile(map_data, (self.newest_files_ncast.forecast_hours[df_index], 1))
                
                # Set map grid data
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 
                                  'latitude'] = time_count[:, 0] = map_data[:, 3]
                self.df_ncast.loc[row_count:row_count + self.newest_files_ncast.data_rows[df_index]-1, 
                                  'longitude'] = time_count[:, 0] = map_data[:, 4]

            # Row count update for new lake
            row_count = row_count + self.newest_files_ncast.data_rows[df_index]     
            
        # -------------------------------------------------------------------------------------------------------------------- #
        # FCAST 
        # -------------------------------------------------------------------------------------------------------------------- #
        
        # Write fcast data to DataFrame
        row_count = 0
        for lake in self.newest_files_fcast.lake.unique():

            df_lake = self.newest_files_fcast[(self.newest_files_fcast.lake == lake)]

            for df_index in df_lake.index:

                # Set attributes to set based on filetype
                if  self.newest_files_fcast.filetype[df_index] == 'WAVES':
                    cols = wave
                elif self.newest_files_fcast.filetype[df_index] == 'WINDS':
                    cols = wind
                elif self.newest_files_fcast.filetype[df_index] == 'SURFACE TEMPS':
                    cols = temperature
                elif self.newest_files_fcast.filetype[df_index] == 'SURFACE CURRENTS':
                    cols = current
                elif self.newest_files_fcast.filetype[df_index] == 'ICE PARAMS':
                    cols = ice   

                # Download text file
                text_file = self.download_file(self.newest_files_fcast.file_url[df_index], 
                                               self.newest_files_fcast.filename[df_index])

                # Get map from header
                map = text_file[0].split()[3].split('/')[-1]  
                map = map.split('.')[0] + '.' + 'map'
                if lake == 'superior':
                    map = 'superior' + map.split('sup')[1]

                # Format text file data as numpy array
                text_data = data_array_vec(text_file)                      # parse rows
                text_data = text_data[text_data != np.array(None)]         # remove headers
                text_data = np.vstack(text_data)

                # Format text file header data as numpy array
                text_header = header_array_vec(text_file)
                text_header = text_header[text_header != np.array(None)]
                text_header = np.vstack(text_header)
                
                print('')
                print('FCAST HEADER')
                print(text_header)
                print('')
                
                # Create empty arrays for time (y, d, r) and datetime (GMT)
                time_count = np.zeros([self.newest_files_fcast.data_rows[df_index] ,3]).astype(int)
                date_time = np.zeros([self.newest_files_fcast.data_rows[df_index] ,1]).astype(object)
                
                # set row count
                count = 0 

                # Loop thought text file headers
                for header in text_header:

                    # Set year, day, hour
                    time_count[count:count + self.newest_files_fcast.grid_count[df_index], 0] = header[0]
                    time_count[count:count + self.newest_files_fcast.grid_count[df_index], 1] = header[1]
                    time_count[count:count + self.newest_files_fcast.grid_count[df_index], 2] = header[2]

                    # Set date time object (GMT)
                    date_time[count:count + self.newest_files_fcast.grid_count[df_index], 0] = datetime.strptime(header[0] + header[1] + header[2], "%Y%j%H") 

                    # Update count
                    count = count + self.newest_files_fcast.grid_count[df_index]

                # Set text file data as DataFrame
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, cols] = text_data

                # Set header year, day, hour
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'year'] = time_count[:, 0]
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'day'] = time_count[:, 1]
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'hour'] = time_count[:, 2]
                
                # Set date time object (GMT)
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'datetime'] = date_time

                # Set static values
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'map'] = map    # map
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 'lake'] = lake  # lake

                # Get map grid data
                map_file = self.download_file(url_map, map)
                map_data = np.loadtxt(map_file)
                map_data = np.tile(map_data, (self.newest_files_fcast.forecast_hours[df_index], 1))
                
                # Set map grid data
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 
                                  'latitude'] = time_count[:, 0] = map_data[:, 3]
                self.df_fcast.loc[row_count:row_count + self.newest_files_fcast.data_rows[df_index]-1, 
                                  'longitude'] = time_count[:, 0] = map_data[:, 4]

            # Row count update for new lake
            row_count = row_count + self.newest_files_fcast.data_rows[df_index]

In [ ]:
# Get all newest files
newest_files = NoaaDB_NewestFile(noaa_files, url_map)

In [ ]:
# Add grid data to newest files DataFrames
newest_files.add_grid_data()

In [ ]:
# Show newest NCAST files
newest_files.newest_files_ncast.head()

In [ ]:
# Show newest FCAST files
newest_files.newest_files_fcast.head()

In [ ]:
newest_files.get_headers()

In [ ]:
newest_files.df_ncast_header

In [ ]:
newest_files.df_fcast_header

In [ ]:
newest_files.df_ncast_header.grid_count[0]*newest_files.df_ncast_header.grid_count[0]

<h1>Step 3</h1>
<h3>Save Data From Newest Files As DataFrame</h3>

In [ ]:
# Setup NCAST and FCAST data DataFrames
newest_files.df_setup()

In [ ]:
# Show NCAST data DataFrame
newest_files.df_ncast.head()

In [ ]:
# Show FCAST data DataFrame
newest_files.df_fcast.head()

In [ ]:
# Fill DataFrames with text file data
newest_files.df_fill()

In [ ]:
# Show NCAST data DataFrame
newest_files.df_ncast.head()

In [ ]:
# Show FCAST data DataFrame
newest_files.df_fcast.head()

<h1>Step 4</h1>
<h3>Load Surf Spots</h3>
<h5>Thanks Grif!</h5>

In [ ]:
# Set surf spots file path and file name
path = r'C:\Users\Sebastian\Projects\Websites\Surfcast\GetData\SurfSpots'
file = 'SurfSpots.csv'

In [ ]:
# Load surf spot data as DataFrame
surf_spots = pd.read_csv(os.path.join(path, file))

In [ ]:
# Show surf spot data
surf_spots.sort('lake')

<h1>Step 5</h1>
<h3>Plot Raw Data</h3>

In [ ]:
# Plot grid points
plt.figure(figsize=(15, 10))

for lake in newest_files.df_ncast.lake.unique():
    
    df = newest_files.df_ncast[(newest_files.df_ncast.lake == lake) & 
                               (newest_files.df_ncast.datetime == newest_files.df_ncast.datetime.unique()[0])] 
    
    plt.plot(-df.longitude, df.latitude, marker='.', linestyle='none', ms=2)

plt.plot(surf_spots.longitude, surf_spots.latitude, color='k', marker='o', linestyle='none', ms=5)    
    
plt.tick_params(labelsize=14)

plt.xlabel('Longitude', fontsize=20)
plt.ylabel('Latitude', fontsize=20)

plt.grid('on')

In [ ]:
# Plot grid points as 3D scatter
attribute = 'wave_height'

plt.figure(figsize=(10, 6))
    
df = newest_files.df_ncast[(newest_files.df_ncast.datetime == newest_files.df_ncast.datetime.unique()[0])] 

sc = plt.scatter(-df.longitude, df.latitude, c = df[attribute], edgecolors='none')

cb = plt.colorbar(sc)
cb.set_label(attribute.replace('_', ' ').title(), fontsize=20)
plt.tick_params(labelsize=14)

plt.xlabel('Longitude', fontsize=20)
plt.ylabel('Latitude', fontsize=20)

plt.grid('on')

<h1>Step 6</h1>
<h3>Create SQL Database</h3>

In [ ]:
# Initializes database with filename 311_8M.db in current directory
surfcast_sql_db = create_engine(r'sqlite:///C:\Users\Sebastian\Projects\Websites\Surfcast\GetData\Surfcast.db')

<h1>Step 7</h1>
<h3>Update SQL Database</h3>

In [ ]:
noaa_files.df.to_sql('data', surfcast_sql_db, if_exists='append')

In [ ]:
df = pd.read_sql_query('SELECT lake FROM data', surfcast_sql_db)
df.head()

In [ ]:
1900-1254.71